<a href="https://colab.research.google.com/github/WilsLoki/test/blob/main/Fine_Tuning_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安装第三方库
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install torch
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 引入模块
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer, # 创建分词器
    AutoConfig,
    AutoModelForSequenceClassification, # 加载初始模型
    DataCollatorWithPadding, # 创建数据整理器
    TrainingArguments, # 定义训练参数
    Trainer) # 创建训练器

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig # 训练模型

import evaluate # 数据预处理_计算准确率 + 评估
import torch # 数据预处理_计算准确率 + 加载未训练模型
import numpy as np # 数据验证

In [ ]:
# 构建数据（训练集+验证集）

imdb_dataset = load_dataset("stanfordnlp/imdb")

# 定义样本大小
# N 表示从数据集中随机抽取的样本数量
N = 1000

# 生成随机索引
# 通过np.random.randint函数，生成了一个大小为N的随机整数数组，这些整数表示从IMDB数据集中随机选取的样本位置
# 训练集包含25000个样本，所以指定索引范围为0到24999
rand_idx = np.random.randint(24999, size=N)

# 提取训练数据（标签+文本）
# imdb_dataset['train'] → 获取训练集数组；[rand_idx] → 从数组中随机选择样本；['text'] → 从选中的样本中只保留文本内容
# 使用上面生成的随机索引（rand_idx），从IMDB数据集的训练集部分（'train'）[ 对应代码imdb_dataset['train'] ]中提取相应的文本数据（'text'）和标签数据（'label'）
# x_train是1000个随机选取的电影评论文本，y_train是对应的标签（0表示负面评论，1表示正面评论）
# 训练数据取1000是为了加速训练，降低对GPU的要求
x_train = imdb_dataset['train'][rand_idx]['text']
y_train = imdb_dataset['train'][rand_idx]['label']

# 提取测试数据（同上）
x_test = imdb_dataset['test'][rand_idx]['text']
y_test = imdb_dataset['test'][rand_idx]['label']

# 创建新的数据集
# 创建一个包含 训练集和验证集 数据的新数据集对象，这个数据集会包含“text”和“label”两列，分别对应文本和标签
# Dataset.from_dict函数用于将数据转换为一个结构化的 Dataset 对象，它能够处理键值对（例如 `'text'` 和 `'label'`），对机器学习友好
dataset = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train}),
                              'validation':Dataset.from_dict({'label':y_test,'text':x_test})})


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# 验证数据
# 计算训练数据集中标签为 1 的比例
# np.array() 表示转换为NumPy数组
# .sum() 表示计算数组中所有数值的总和，由于 label 只有 0 和 1，求和就等于训练集中“正面评价”的数量
# .len() 表示计算数据综述
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

0.506

In [ ]:
# 加载初始模型

# 选择预训练模型
# 只是定义了一个字符串变量，相当于存储了模型的名称，它并不会触发下载
model_checkpoint = 'distilbert-base-uncased'

# 定义标签映射
# WHY:模型内部使用数字(0,1)处理数据，但在展示结果时,我们希望看到有意义的文本(Negative/Positive)，这两个字典实现了数字和文本之间的相互转换
id2label = {0: "Negative", 1: "Positive"} # 用于将模型输出的数字预测(0,1)转换为可读的文本(Negative/Positive)
label2id = {"Negative":0, "Positive":1} # 用于将文本标签(Negative/Positive)转换为模型训练需要的数字(0,1)

# 创建模型
# AutoModelForSequenceClassification 是一个用于文本分类的专用模型【类】，它会根据你提供的预训练模型自动选择合适的模型架构，具体来说，1. 识别这是DistilBERT模型；2. 加载对应的模型架构（DistilBertForSequenceClassification）；3. 设置正确的注意力机制和层数；4. 配置适合的分词器和预处理步骤；5. 加载预训练权重到正确的层
# from_pretrained()是一个类的方法，具体来说，1.下载预训练模型；2.加载模型结构和权重；3.应用自定义参数配置
# 总结：先由AutoModelForSequenceClassification 识别并选择正确的模型架构；然后from_pretrained准备和加载自定义参数的模型
# 说明：此处的AutoModelForSequenceClassification.from_pretrained()是一个静态方法（Static Method），区别于实例方法（Instance Method），它类似于普通函数，只是放在类中，通常用于工具性功能
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, # 使用之前选择的预训练模型
    num_labels=2, # 告诉模型需要将文本分成几类(此处为二分类)
    id2label=id2label, # 告诉模型如何在数字和文本标签之间转换
    label2id=label2id
    )

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 显示模型架构
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
# 创建分词器（前置条件）
# AutoTokenizer 的作用是将文本进行分词（Tokenization），并将每个词或标记映射为对应的数字 ID，这些 ID 是根据模型的词汇表（vocabulary）来映射的
# Embedding 是通过模型的嵌入层（embedding layer）将这些数字 ID 转换为高维向量，进而捕捉词语的语义
# add_prefix_space=True：用于添加前导空格，有助于区分连续词汇
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# 添加填充标记
# 如果分词器没有定义填充标记（pad_token），则添加一个 [PAD] 标记，并且调整模型的词汇表大小，以适应新的标记
# 填充标记（pad_token）用于将不同长度的输入文本统一为相同的长度，以便批处理（batch processing），常见的填充标记包括 [PAD]或 0
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# 定义分词函数（核心部分）

def tokenize_function(examples): # 参数为examples

    # 提取文本
    text = examples["text"] # examples为字典

    # 设置截断方向为左侧
    # 如果文本太长需要截断，会从左边开始截断，这意味着保留文本的右侧部分，因为在情感分析中，句子的后半部分通常包含更重要的情感信息
    tokenizer.truncation_side = "left"

    # 分词处理
    tokenized_inputs = tokenizer(
        text, # 输入文本
        return_tensors="np", # 指定返回 numpy 数组格式
        truncation=True, # 启用截断
        max_length=512,  # 设置序列的最大长度为512，超过这个长度的文本会被截断
        padding='max_length' # 对短文本进行填充，填充到max_length指定的长度，使所有文本长度一致
    )

    # 返回统一长度的numpy数组
    return tokenized_inputs

In [ ]:
# 对训练集和验证集执行分词处理（执行操作）

# 对数据集中的每一个文本样本应用 tokenize_function
# dataset为刚才创建的数据集
# .map为数据集的一个方法，用于对数据集中的每个元素应用一个函数
# tokenize_function为刚才定义的分词函数
# batched=True: 启用批处理模式，即，批量处理数据集，而不是一次处理一个样本，其优势为比逐条处理拥有更快的处理速度
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 返回新的处理过的结果
tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
# 计算模型在给定数据集上的准确率

# 【metrics并没有使用，可能为冗余】加载准确率评估指标
# evaluate 是 Hugging Face 提供的一个评估库，提供了多种常见的评估指标，比如准确率、精确率、召回率等
# load("accuracy") 函数会加载一个预定义的准确率计算器
# 变量名为 metrics 是为了和 评估部分 的变量accuracy区分开，避免冲突
metrics = evaluate.load("accuracy")

# 定义计算准确率的函数（评估时计算准确率）
def compute_accuracy(model, dataset, tokenizer): # model: 训练好的模型；dataset: 用于评估的数据集；tokenizer: 分词器

    # 将模型设置为评估模式（区别于训练模式）（关闭 dropout 层，防止随机丢弃神经元；固定 batch normalization 层的参数）
    model.eval()

    # 初始化准确率评估指标 accuracy metric
    accuracy_metric = evaluate.load("accuracy")

    # 从dataset中提取输入ID、注意力掩码和标签，并转换为PyTorch张量（tensor）
    # PyTorch的基本数据结构是张量（tensor），相比普通的Python列表或NumPy数组，其在深度学习中计算更方便
    # 第1步：获取数据，dataset['input_ids']：从dataset字典中获取'input_ids'对应的值
    # 第2步：转换为张量，torch.tensor(): 将输入数据转换为PyTorch张量（tensor）
    # 第3步：将张量移动到指定设备，.to(model.device)，其中 model.device表示获取模型所在的设备类型
    input_ids = torch.tensor(dataset['input_ids']).to(model.device) # input_ids：分词器数字化结果
    attention_mask = torch.tensor(dataset['attention_mask']).to(model.device) # attention_mask: 标记哪些位置是真实token，哪些是填充
    labels = torch.tensor(dataset['label']).to(model.device) #

    # 不计算梯度，在推理和评估时，不需要计算梯度，因此禁用它以提升计算效率
    # 在 with 语句块内，PyTorch 的所有操作都不会记录梯度，离开 with 语句块后，梯度计算会恢复正常。它的使用类似于文件操作中的 with 语句，保证在执行块内的代码时不会有不必要的副作用
    with torch.no_grad():
        # 获取模型输出
        # 将数据input_ids 和 attention_mask输入到模型中，进行前向传播（inference）
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        # 获取预测概率
        # outputs.logits 通过访问模型返回的 outputs 对象中的 logits 属性，提取出模型的原始输出
        # logits 是 未经过激活函数处理的原始输出。激活函数（如 Softmax 或 Sigmoid）会将这些分数转换为 概率 或 类别的可信度
        logits = outputs.logits
        # （通过寻找每个样本的最大logits）获取最可能的类别
        # torch.argmax() 是一个 PyTorch 函数，用于返回指定维度上的最大值的索引
        # dim=1 表示 按行操作（在每一行上寻找最大值的索引）
        predictions = torch.argmax(logits, dim=1)

        # 更新准确率评估指标 accuracy metric
        # accuracy_metric.add_batch() 会将当前的预测结果（predictions）和真实标签（labels） 添加 到准确率评估中
        accuracy_metric.add_batch(predictions=predictions, references=labels)

    # 计算并返回最终准确率
    # 首先调用 compute() 方法来计算准确率，它的返回值是一个字典，如{"accuracy": 0.85}；接着从计算结果中提取 "accuracy" 对应的值，并将其赋给变量 accuracy
    accuracy = accuracy_metric.compute()["accuracy"]
    return accuracy

# 计算验证集上的准确率
subset_size = 100  # 设置子集大小为100，从验证集（validation）中随机选择 100 个样本进行准确率评估
subset_indices = range(subset_size) # 使用 range(subset_size) 来生成一个整数序列，范围从 0 到 99（共 100 个数字），这些整数将作为 索引
# 从 tokenized_dataset 中的 "validation" 数据集部分，根据前面生成的 subset_indices 选择出 前 100 个样本（此处为固定选择了验证集中的前100个样本，也可以通过random_indices生成随机索引，从而选择随机样本）
subset_data = tokenized_dataset["validation"].select(subset_indices)
accuracy_result = compute_accuracy(model, subset_data, tokenizer) # 调用了之前定义的 compute_accuracy 函数，来计算模型在 subset_data（即前 100 个样本）上的准确率
print("Validation Accuracy:", accuracy_result)

Validation Accuracy: 0.46


In [ ]:
# 重新初始化模型（为了确保从头开始训练）【此处重新初始化的原因待定，可能为冗余】
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 创建数据整理器
# DataCollatorWithPadding 在 批处理（训练的一个环节） 时，动态填充每个批次中的文本，使其填充到批次中最长的文本长度
# 而 tokenize_function 中 padding='max_length'，则是在 文本预处理 时，将每个文本填充到 固定长度
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# 导入评估模块

# 加载准确率评估指标
accuracy = evaluate.load("accuracy")

# 定义评估函数（训练中计算准确率）
def compute_metrics(p):
    # p为元组，以下代码为元组解包（tuple unpacking），等同于
    # predictions = p[0] 获取元组的第一个元素（模型预测值）
    # labels = p[1] 获取元组的第二个元素（真实标签）
    # 在后续训练模型part，Trainer会自动：1.收集一个批次的模型预测结果；2.将这些预测结果和对应的真实标签打包成一个元组；3.将这个元组作为参数p传入compute_metrics函数；4.使用返回的评估结果（准确率）来监控训练进度
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1) # 获取预测标签

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)} # 返回准确率


In [ ]:
# 加载未训练的模型，用于对比
model_untrained = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

# 测试几个示例文本
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

# 使用未训练模型进行预测
print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # 将文本分词并转换为PyTorch tensor格式，其中，
    # tokenizer.encode(text) 分词 + 数字化
    # return_tensors="pt" 使得返回的结果是一个 PyTorch tensor
    inputs = tokenizer.encode(text, return_tensors="pt")
    # 计算 logits
    logits = model_untrained(inputs).logits
    # 将 logits 转化为 label
    predictions = torch.argmax(logits)
    # 打印预测结果
    # predictions 是一个 PyTorch tensor
    # tolist() 是 PyTorch tensor 对象的一个方法，用来将张量（tensor）转换为 Python 列表（list）格式
    # id2label 是一个字典，用来将类别 ID 映射到对应的标签
    # 示例：Better than the first one. - Positive
    print(text + " - " + id2label[predictions.tolist()])

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Untrained model predictions:
----------------------------
It was good. - Positive
Not a fan, don't recommed. - Positive
Better than the first one. - Positive
This is not worth watching even once. - Negative
This one is a pass. - Positive


In [ ]:
# 训练模型

# 配置LoRA参数
peft_config = LoraConfig(task_type="SEQ_CLS", # 任务类型是序列分类（Sequence Classification）
                        r=16, # LoRA的秩，其值越小，模型微调时参数更新越少
                        lora_alpha=32, # LoRA的缩放参数，其控制了LoRA的影响力。如果该值过大，LoRA会强烈影响原模型的权重更新
                        lora_dropout=0.05, # LoRA的dropout率，即在训练过程中随机“丢弃”神经元的激活值，目的是减少模型对训练数据的依赖，提高模型在未见数据上的表现
                        target_modules = ['q_lin', 'v_lin']) # 要应用LoRA的模块

print(f"lora_dropout的值是: {peft_config.lora_dropout}")
print(f"target_modules的值是: {peft_config.target_modules}")

lora_dropout的值是: 0.05
target_modules的值是: {'v_lin', 'q_lin'}


In [ ]:
# 打印配置对象 peft_config，确认配置是否正确
peft_config

LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=16, target_modules={'v_lin', 'q_lin'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [ ]:
for name, module in model.named_modules():
    print(name)


distilbert
distilbert.embeddings
distilbert.embeddings.word_embeddings
distilbert.embeddings.position_embeddings
distilbert.embeddings.LayerNorm
distilbert.embeddings.dropout
distilbert.transformer
distilbert.transformer.layer
distilbert.transformer.layer.0
distilbert.transformer.layer.0.attention
distilbert.transformer.layer.0.attention.dropout
distilbert.transformer.layer.0.attention.q_lin
distilbert.transformer.layer.0.attention.k_lin
distilbert.transformer.layer.0.attention.v_lin
distilbert.transformer.layer.0.attention.out_lin
distilbert.transformer.layer.0.sa_layer_norm
distilbert.transformer.layer.0.ffn
distilbert.transformer.layer.0.ffn.dropout
distilbert.transformer.layer.0.ffn.lin1
distilbert.transformer.layer.0.ffn.lin2
distilbert.transformer.layer.0.ffn.activation
distilbert.transformer.layer.0.output_layer_norm
distilbert.transformer.layer.1
distilbert.transformer.layer.1.attention
distilbert.transformer.layer.1.attention.dropout
distilbert.transformer.layer.1.attention.q

In [ ]:
# 将模型与LoRA配置结合，生成经过 LoRA 微调配置后的模型
# get_peft_model：用于将指定的模型（model）与 LoRA 配置（peft_config）结合，生成一个经过 LoRA 微调配置后的模型
# model：刚才加载的预训练模型
# peft_config：是一个 LoraConfig 配置对象，包含了所有 LoRA 微调的参数（例如秩、目标模块、alpha 等）。
model = get_peft_model(model, peft_config)
model.print_trainable_parameters() # 打印可训练的参数数量，trainable_parameters 代表的是 target_modules 中定义的模块

trainable params: 887,042 || all params: 67,842,052 || trainable%: 1.3075


In [ ]:
# 设置训练超参数【可尝试不同的超参数调整效果】
lr = 1e-3 # 学习率，即0.001，控制模型在训练时参数更新的步长
batch_size = 4 # 批次大小，控制每次更新时处理的样本数量。较大的批次：训练更稳定，但需要更多内存
num_epochs = 10 # 训练轮次，控制训练数据被完整处理的次数

In [ ]:
# 定义训练参数
training_args = TrainingArguments( # TrainingArguments 是一个用来配置模型训练的参数类，涵盖了学习率、批处理大小、评估策略、模型保存策略等
    output_dir= model_checkpoint + "-lora-text-classification", # 输出目录，（通过新建文件夹）指定模型和训练结果保存的目录
    learning_rate=lr, # 学习率
    #warmup_ratio = 0,
    #lr_scheduler_type = "cosine",
    per_device_train_batch_size=batch_size, # 训练批次大小
    per_device_eval_batch_size=batch_size, # 评估批次大小
    num_train_epochs=num_epochs, # 训练轮数
    weight_decay=0.03, # 权重衰减，防止模型过拟合的正则化技术
    evaluation_strategy="epoch",  # 评估策略，决定何时在验证集上评估模型，此处为每轮结束后评估
    save_strategy="epoch",  # 保存策略，决定何时保存模型检查点，此处为每轮保存模型
    load_best_model_at_end=True,  # 加载最佳模型，即训练结束后加载验证集表现最好的模型
)

# 在定义参数后打印学习率，确认修改是否生效
print(f"Learning Rate: {training_args.learning_rate}")
#print(f"Warm up Ratio: {training_args.warmup_ratio}")
#print(f"Learning Rate Scheduler Type: {training_args.lr_scheduler_type}")
print(f"Weight Decay: {training_args.weight_decay}")
print(f"Train Epochs: {training_args.num_train_epochs}")
print(f"Batch Size: {training_args.per_device_train_batch_size}")

Learning Rate: 0.001
Weight Decay: 0.03
Train Epochs: 10
Batch Size: 4


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# 创建训练器
# 在 Hugging Face Trainer 中 默认启用了 W&B 日志功能，可通过 report_to="none" 禁用
trainer = Trainer( # Trainer是Hugging Face transformers库中的一个核心类，它封装了整个训练过程
    model=model, # 传入之前创建的模型
    args=training_args, # 之前定义的训练参数
    train_dataset=tokenized_dataset["train"], # 训练数据集，包含已经分词的文本（input_ids, input_ids）和标签(label)
    eval_dataset=tokenized_dataset["validation"], # 验证数据集，包含已经分词的文本和标签
    tokenizer=tokenizer, # 分词器
    data_collator=data_collator, # 数据整理器
    compute_metrics=compute_metrics, # 评估函数
)

# 开始训练
trainer.train()

# 输出分析
# 训练损失（Training Loss）：是一个度量模型预测与实际标签之间误差的值。训练损失越小，表示模型在训练集上越拟合得好
# 验证损失（Validation Loss）：计算方式同训练损失，其作用是评估模型的 泛化能力，即模型在新数据上的表现如何
# 两者结合起来，可以帮助我们了解模型是否过拟合、是否有良好的泛化能力，并指导我们进一步调整模型和训练过程，如选择合适的超参数，提前停止训练，模型调优等

# 训练步数（global_step）：指模型执行的 梯度更新（即参数更新）次数。每执行一次梯度更新，都会通过前向传播（计算预测）、损失函数（计算误差）以及反向传播（计算梯度并更新参数）来调整模型的参数
# 训练步数 = 训练集中样本数 * epochs / batch size

# 浮点运算数（FLOP）：用于衡量计算任务的计算复杂度，通常表示执行浮点运算的总次数

<ipython-input-25-7b085eaabafd>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer( # Trainer是Hugging Face transformers库中的一个核心类，它封装了整个训练过程


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.727846,{'accuracy': 0.833}
2,0.358000,0.626812,{'accuracy': 0.852}
3,0.358000,0.656787,{'accuracy': 0.872}
4,0.187300,0.766267,{'accuracy': 0.883}
5,0.187300,0.770388,{'accuracy': 0.877}
6,0.043700,0.898124,{'accuracy': 0.893}
7,0.043700,0.987155,{'accuracy': 0.886}
8,0.014800,1.002246,{'accuracy': 0.888}
9,0.014800,1.047115,{'accuracy': 0.892}
10,0.000600,1.033514,{'accuracy': 0.889}


Trainer is attempting to log a value of "{'accuracy': 0.833}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.852}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.872}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.883}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.877}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This i

TrainOutput(global_step=2500, training_loss=0.12089313198328018, metrics={'train_runtime': 526.0886, 'train_samples_per_second': 19.008, 'train_steps_per_second': 4.752, 'total_flos': 1351923916800000.0, 'train_loss': 0.12089313198328018, 'epoch': 10.0})

In [ ]:
# 生成预测

model.to('cpu') # 将模型 model 移动到 CPU 上运行。

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    # 步骤1：准备输入数据，包括分词+数字化，转化为python tensor，并将数据移动到 CPU 上进行计算
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu")
    # 步骤2：获取模型预测
    logits = model(inputs).logits
    # 步骤3：处理预测结果
    # torch.max() 用于获取张量中的最大值
    predictions = torch.max(logits,1).indices # torch.max(input, dim)会返回两个结果，此处返回最大值和最大值的索引（表示模型预测的类别）
    # 步骤4：转换预测结果并打印
    # predictions.tolist()[0] 将张量转换为Python列表并获取第一个元素
    # predictions是之前在compute_metrics函数中定义的变量，其中具体自定义了predictions, labels = p，即predictions = p[0]，labels = p[1]，所以此处predictions.tolist()[0]]而不是[1]
    # 输出示例：Not a fan, don't recommend. - Negative
    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
It was good. - Positive
Not a fan, don't recommed. - Negative
Better than the first one. - Positive
This is not worth watching even once. - Negative
This one is a pass. - Negative


In [ ]:
# 登录到Hugging Face Hub
# 通过notebook_login函数，提供access token，实现登录
from huggingface_hub import notebook_login
# 这行代码会弹出一个登录提示，要求你输入 Hugging Face 的访问令牌（token）
notebook_login() # ensure token gives write access

In [ ]:
# 上传模型至 Hugging Face Hub
hf_name = 'WillLoki' # 需要替换成你自己的 Hugging Face 用户名或组织名
# model_checkpoint = 'distilbert-base-uncased'
model_id = hf_name + "/" + model_checkpoint + "-lora-text-classification" # 模型ID，可自定义
# push_to_hub() 会将模型上传到 Hugging Face Hub，其会保存模型权重，模型配置，分词器配置，训练参数，和其他必要的文件
model.push_to_hub(model_id) # 保存模型
trainer.push_to_hub(model_id) # 保存训练器

# Why上传模型至Hugging Face: 1.方便模型共享；2.便于版本管理；3.便于模型在训练和推理；4.自动化部署，直接在线推理；5.方便跟踪和监控

# 输出分析
# adapter_model.safetensors：上传文件，其中包含了LoRA的低秩矩阵参数。命名中含有adapter是因为在PEFT库中，所有的参数高效微调方法（包括LoRA）都被统一称为"adapters"，这是一个通用术语，用来描述任何形式的模型参数修改或附加，而不是特指传统意义上的adapter layers
# events.out.tfevents 是 TensorBoard 事件文件，通常包含训练过程中的日志数据，如训练损失、准确率等
# training_args.bin 是包含训练参数的二进制文件
# repo_url=RepoUrl('https://huggingface.co/WillLoki/distilbert-base-uncased-lora-text-classification', 为上传的模型仓库的 URL

README.md:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.55M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1741057738.7157d4ee19a0.1159.1:   0%|          | 0.00/9.32k [00:00<?, ?B/s]

events.out.tfevents.1741056335.7157d4ee19a0.1159.0:   0%|          | 0.00/9.27k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/WillLoki/distilbert-base-uncased-lora-text-classification/commit/1a1d3ed70ecf207e9107697e1c31bfacc63795c4', commit_message='WillLoki/distilbert-base-uncased-lora-text-classification', commit_description='', oid='1a1d3ed70ecf207e9107697e1c31bfacc63795c4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/WillLoki/distilbert-base-uncased-lora-text-classification', endpoint='https://huggingface.co', repo_type='model', repo_id='WillLoki/distilbert-base-uncased-lora-text-classification'), pr_revision=None, pr_num=None)

In [ ]:
# 从 Hugging Face Hub 加载模型配置

# 加载模型的配置文件
config = PeftConfig.from_pretrained(model_id)
# 加载模型
# config.base_model_name_or_path 是从上一步加载的配置文件中获取的模型名称或路径，它指向了预训练模型的位置
inference_model = AutoModelForSequenceClassification.from_pretrained(
    config.base_model_name_or_path, num_labels=2, id2label=id2label, label2id=label2id
)
# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# 加载LoRA微调后的模型（通过PeftModel加载）
model = PeftModel.from_pretrained(inference_model, model_id)

# 输出分析
# Some weights of DistilBertForSequenceClassification were not initialized...：这是正常的，因为提到的这些层是为分类任务专门添加的，在原始的预训练模型中并不存在

adapter_config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/3.55M [00:00<?, ?B/s]

In [ ]:
# 模型推理
# 用来验证上传到 Hugging Face 后的模型是否能够正确地进行推理
print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu")
    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
It was good. - Positive
Not a fan, don't recommed. - Negative
Better than the first one. - Positive
This is not worth watching even once. - Negative
This one is a pass. - Positive
